In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
                              # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)

In [2]:
# 카테고리에 대한 주소를 반복문을 통해 얻는 것 보다 카테고리 주소를 직접 저장하는 것이 간편하여 실행
base_url = 'http://ota9.co.kr'

top = 'http://ota9.co.kr/product/list.html?cate_no=127'
bottom = 'http://ota9.co.kr/product/list.html?cate_no=128'
outer = 'http://ota9.co.kr/product/list.html?cate_no=25'
training = 'http://ota9.co.kr/product/list.html?cate_no=27'

category_list_url = [top, bottom, outer, training]

In [3]:
list = pd.DataFrame({'color_view' : [], 'size' : []}) # 데이터를 추가하기 위해 list라는 데이터프레임을 미리 저장

page_url = category_list_url[2] + '&page=' + '{}' 
# 옷타쿠 또한 페이지수가 넘어 갈수록 끝에 &page=1 , &page=2 , &page=3 형식으로 일정한 패턴을 가진다.

page_1 = requests.get(category_list_url[2], headers = {'user-agent' : 
                                                       ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                       'Chrome/71.0.3578.98 Safari/537.36')}) # 유저에이전트를 설정해서 접근가능하게 한다.
page_2 = BeautifulSoup(page_1.content, 'html.parser')
page_3 = page_2.find('div', {'class' : 'xans-element- xans-product xans-product-normalpaging'})
page_4 = page_3.find_all('li', {'class' : 'xans-record-'})
page_5 = len(page_4)

for page in range(page_5) : # 각 페이지를 반복해서 데이터를 수집하기 위한 반복문
    res = requests.get(page_url.format(str(page)), 
                       headers = {'user-agent' : 
                                  ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                   'Chrome/71.0.3578.98 Safari/537.36')})
    html = BeautifulSoup(res.content, 'html.parser') # html 소스 획득
    table = html.find('ul', { 'class': 'prdList column4' }) 
    # 'item xans-record-' 클래스가 여러개였기 때문에 지정된 클래스를 가지고 오기 위해 상위 클래스를 먼저 가지고 옴.
    table1 = table.find_all('li', {'class' : 'item xans-record-'}) # 각각의 상품에 접근하기 위해 클래스획득
        
    products = []
    for i in range(len(table1)) :
        prod = table1[i].find('a')['href'] # 카테고리의 href를 가지고 온것 처럼 한 카테고리의 한 상품에 대한 href를 가지고 오는 것
        products.append(prod)
        
    url_list = []
    for i in range(len(products)) :
        url = base_url + products[i] # 이를 base_url과 합쳐서 각각의 상품에 접근할 수 있는 url을 획득한다.
        url_list.append(url)
        
    for product_url in url_list :
        res = requests.get(product_url, headers = {'user-agent' : 
                                                   ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                       'Chrome/71.0.3578.98 Safari/537.36')}) # 주소획득
        html = BeautifulSoup(res.content, 'html.parser') # html 소스 획득
        table = html.find('div', { 'class': 'cont' }) # 각 상품의 이미지를 추출하기 위한 클래스획득
        img = table.find_all('img')
        
        color_view = []
        size_table = []
        
        src = []
        for i in range(len(img)) :
            img_src = img[i]['src'] # 이미지의 주소에 대한 정보는 img 태그 내의 src 속성에 있다. 이를 가지고 오는 것.
            src.append(img_src)
     
 # 각 상품에 대한 color_view(옷 이미지), size(사이즈표) 를 추출하기 위한 반복문 구문인데, 
 # 오타9의 페이지 소스에는 옷 이미지에 해당하는 곳에 top 이라는 단어가 있고,
 # 사이즈표 이미지에 해당하는 곳에 size 라는 단어가 존재하므로 이를 통해 옷이미지와 사이즈표를 추출하였다.  
    
        for result in src :
            if "top" in result :
                x = result 
                if x not in color_view :
                    color_view.append(x)
            
            elif "size" in result :
                x = result
                if x not in size_table :
                    size_table.append(x)
        
            else :
                pass
            
        if color_view and size_table : # 각각의 옷이미지와 사이즈표가 두 가지 이상 있는 상품이 있으므로 필요한 하나만을 추출.
            cv = color_view[0]
            sz = size_table[0]
            df = pd.DataFrame({'color_view' :[cv], 'size' : [sz]})
            list = list.append(df) # 데이터 프레임 형식으로 데이터를 추가

In [4]:
list

,color_view,size
0,/web/product/page/pa-mr-cotton-m-top.jpg,/web/product/page/pa-mr-cotton-m-size.jpg
0,/web/product/page/pa-mf-325-ck-m-top.jpg,/web/product/page/pa-mf-325-ck-m-size.jpg
0,/web/product/page/vs-mf-wool-ame-m-top.jpg,/web/product/page/vs-mf-wool-ame-m-size.jpg
0,/web/product/page/hz-bs-pleese-m-top.jpg,/web/product/page/hz-bs-pleese-m-size.jpg
0,/web/product/page/jp-bh-neo-padd-m-top.jpg,/web/product/page/jp-bh-neo-padd-m-size.jpg
0,/web/product/page/jp-dda-dumble-padd-m-top.jpg,/web/product/page/jp-dda-dumble-padd-m-size.jpg
0,/web/product/page/jp-jm-come-down-m-top.jpg,/web/product/page/jp-jm-come-down-m-size.jpg
0,/web/product/page/co-bh-check-wool-db-m-top.jpg,/web/product/page/co-bh-check-wool-db-m-size.jpg
0,/web/product/page/jp-bh-codu-nubim-m-top.jpg,/web/product/page/jp-bh-codu-nubim-m-size.jpg
0,/web/product/page/pa-dda-mask-hood-m-top.jpg,/web/product/page/pa-dda-mask-hood-m-size.jpg


In [5]:
list1 = base_url + list # base_url을 결합해주어 완전한 형태의 url을 만들어 준다.

In [6]:
list1

,color_view,size
0,http://ota9.co.kr/web/product/page/pa-mr-cotto...,http://ota9.co.kr/web/product/page/pa-mr-cotto...
0,http://ota9.co.kr/web/product/page/pa-mf-325-c...,http://ota9.co.kr/web/product/page/pa-mf-325-c...
0,http://ota9.co.kr/web/product/page/vs-mf-wool-...,http://ota9.co.kr/web/product/page/vs-mf-wool-...
0,http://ota9.co.kr/web/product/page/hz-bs-plees...,http://ota9.co.kr/web/product/page/hz-bs-plees...
0,http://ota9.co.kr/web/product/page/jp-bh-neo-p...,http://ota9.co.kr/web/product/page/jp-bh-neo-p...
0,http://ota9.co.kr/web/product/page/jp-dda-dumb...,http://ota9.co.kr/web/product/page/jp-dda-dumb...
0,http://ota9.co.kr/web/product/page/jp-jm-come-...,http://ota9.co.kr/web/product/page/jp-jm-come-...
0,http://ota9.co.kr/web/product/page/co-bh-check...,http://ota9.co.kr/web/product/page/co-bh-check...
0,http://ota9.co.kr/web/product/page/jp-bh-codu-...,http://ota9.co.kr/web/product/page/jp-bh-codu-...
0,http://ota9.co.kr/web/product/page/pa-dda-mask...,http://ota9.co.kr/web/product/page/pa-dda-mask...


In [7]:
list1.index = range(len(list1)) # 데이터를 append한 것이기 때문에 인덱스가 엉망이므로, 재설정을 해줌.

In [13]:
list2 = list1.drop(331,axis=0) # 몇개의 상품에서 이미지파일을 추출할 수 없는 에러가 발생.
                               # 반복문을 통해 예외를 처리할 수도 있었지만, 몇개 되지 않아 일일히 제거해줌.

In [14]:
list2.index = range(len(list2))

In [15]:
list2

,color_view,size
0,http://ota9.co.kr/web/product/page/pa-mr-cotto...,http://ota9.co.kr/web/product/page/pa-mr-cotto...
1,http://ota9.co.kr/web/product/page/pa-mf-325-c...,http://ota9.co.kr/web/product/page/pa-mf-325-c...
2,http://ota9.co.kr/web/product/page/vs-mf-wool-...,http://ota9.co.kr/web/product/page/vs-mf-wool-...
3,http://ota9.co.kr/web/product/page/hz-bs-plees...,http://ota9.co.kr/web/product/page/hz-bs-plees...
4,http://ota9.co.kr/web/product/page/jp-bh-neo-p...,http://ota9.co.kr/web/product/page/jp-bh-neo-p...
5,http://ota9.co.kr/web/product/page/jp-dda-dumb...,http://ota9.co.kr/web/product/page/jp-dda-dumb...
6,http://ota9.co.kr/web/product/page/jp-jm-come-...,http://ota9.co.kr/web/product/page/jp-jm-come-...
7,http://ota9.co.kr/web/product/page/co-bh-check...,http://ota9.co.kr/web/product/page/co-bh-check...
8,http://ota9.co.kr/web/product/page/jp-bh-codu-...,http://ota9.co.kr/web/product/page/jp-bh-codu-...
9,http://ota9.co.kr/web/product/page/pa-dda-mask...,http://ota9.co.kr/web/product/page/pa-dda-mask...


In [25]:
list3 = list2.drop([96,215], axis=0)

In [26]:
list3.index = range(len(list3)) # 최종적으로 인덱스 설정

In [27]:
list3

,color_view,size
0,http://ota9.co.kr/web/product/page/pa-mr-cotto...,http://ota9.co.kr/web/product/page/pa-mr-cotto...
1,http://ota9.co.kr/web/product/page/pa-mf-325-c...,http://ota9.co.kr/web/product/page/pa-mf-325-c...
2,http://ota9.co.kr/web/product/page/vs-mf-wool-...,http://ota9.co.kr/web/product/page/vs-mf-wool-...
3,http://ota9.co.kr/web/product/page/hz-bs-plees...,http://ota9.co.kr/web/product/page/hz-bs-plees...
4,http://ota9.co.kr/web/product/page/jp-bh-neo-p...,http://ota9.co.kr/web/product/page/jp-bh-neo-p...
5,http://ota9.co.kr/web/product/page/jp-dda-dumb...,http://ota9.co.kr/web/product/page/jp-dda-dumb...
6,http://ota9.co.kr/web/product/page/jp-jm-come-...,http://ota9.co.kr/web/product/page/jp-jm-come-...
7,http://ota9.co.kr/web/product/page/co-bh-check...,http://ota9.co.kr/web/product/page/co-bh-check...
8,http://ota9.co.kr/web/product/page/jp-bh-codu-...,http://ota9.co.kr/web/product/page/jp-bh-codu-...
9,http://ota9.co.kr/web/product/page/pa-dda-mask...,http://ota9.co.kr/web/product/page/pa-dda-mask...


In [28]:
import urllib.request # 이미지 파일을 저장하기 위한 패키지, 웹사이트의 이미지 주소가 있으면 다운 받을 수 있다.

In [29]:
try :
    for i in range(len(list3)) : # 사이즈 표에 대한 이미지 파일을 저장하는 것, 반복문을 통해 하나씩 저장해주었다.
        url = list3['size'].iloc[i]
        name = 'outer' + 'size' + str(i) + '.jpg' # ex) 아우터size100.jpg 형식으로 파일을 저장
        urllib.request.urlretrieve(url, name)
        
except urllib.error.HTTPError :
    pass

In [ ]:
try :
    for i in range(84, len(list3)) : # 옷 이미지에 대한 이미지 파일을 저장하는 것, 반복문을 통해 하나씩 저장해주었다.
        url = list3['color_view'].iloc[i]
        name = 'outer' + 'color_view' + str(i) + '.jpg' # ex) 아우터color_view100.jpg 형식으로 파일을 저장하였다.
        urllib.request.urlretrieve(url, name)
        
except urllib.error.HTTPError :
    pass